In [15]:
import os
import json
from openai import OpenAI
from datetime import datetime, timedelta
from dotenv import load_dotenv
from core.messages_service import MessagesService

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [16]:
# Initialize conversation history
messages_service = MessagesService()
user_interest = None

# Initial system message
system_prompt = "You are an assistant that helps users find sushi restaurants and parking spots in Munich. When a user expresses interest in either sushi, parking, or both, use the set_user_interest function to record their preference."
messages_service.add_system_message(system_prompt)

# user_prompt = "I am Hungry"
user_prompt = input("eg. I am looking for a Sushi Restaurant in Munich.")
messages_service.add_user_message(user_prompt)

messages_service.print_messages()

{'role': 'system', 'content': 'You are an assistant that helps users find sushi restaurants and parking spots in Munich. When a user expresses interest in either sushi, parking, or both, use the set_user_interest function to record their preference.'}
{'role': 'user', 'content': ''}


In [17]:
from tools.user_interest_tools import user_interest_tools
from tools.parking_tools import parking_tools
from tools.sushi_tools import sushi_tools

In [18]:
tool_choice = {
    "type": "function",
    "function": {"name": "get_user_interest"}
}

In [19]:
# Get user interest
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages_service.get_messages(),
    tool_choice=tool_choice,
    tools=user_interest_tools,
)

assistant_message = completion.choices[0].message
print(assistant_message)


# The json.loads function converts the string to a Python object
from services.user_interest_service import get_user_interest
from services.enum_encoder import EnumEncoder

interest = json.loads(assistant_message.tool_calls[0].function.arguments).get("interest")
print(interest)

# Call the function with arguments
chosen_function = eval(assistant_message.tool_calls[0].function.name)
user_interest = chosen_function(interest)

user_interest_json = json.dumps(user_interest, cls=EnumEncoder)
print(user_interest_json)

messages_service.add_function_message(assistant_message.tool_calls[0].function.name, user_interest_json)
messages_service.print_last_message()


ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XML6TghyukrJODy7sUCzW3Gu', function=Function(arguments='{"interest":"BOTH"}', name='get_user_interest'), type='function')])
BOTH
["Sasou", "Galeria Restaurant", "Shaokao Asian Grill&Wine", "Secret Garden", "Marienplatz", "City", "Platzl Hotel M\u00fcnchen"]
{'role': 'function', 'name': 'get_user_interest', 'content': '["Sasou", "Galeria Restaurant", "Shaokao Asian Grill&Wine", "Secret Garden", "Marienplatz", "City", "Platzl Hotel M\\u00fcnchen"]'}


In [20]:
def call_function(assistant_message):
    for tool_call in assistant_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        # Here you would actually call your function and get the result
        # For now, we'll just create a mock response
        function_response = {"status": "success", "interest": function_args.get("interest")}

In [21]:
# completion = client.chat.completions.create(
#     model="gpt-4o",
#     messages=[{"role": "user", "content": "What's the weather like in Paris today?"}],
#     tools=sushi_tools,
# )